In [11]:
import sys
import pandas as pd
import numpy as np
import datetime as dt
import glob

# 프로젝트 루트 직접 지정
sys.path.insert(0, r"C:\airflow")

# 코드

In [12]:
import glob
import pandas as pd

coupang_glob = glob.glob(r"C:\Users\민준\OneDrive - 주식회사 도리당\Collect_Data\영업팀\coupangeats_orders*.csv")


coupang_df = pd.concat(
    map(pd.read_csv, coupang_glob),
    ignore_index=True
)

coupang_df.head()


,collected_at,store_id,store_name,order_date,order_id,delivery_type,order_status,order_summary,total_price,is_cancelled,...,매출액,상점부담_쿠폰,중개_이용료,결제대행사_수수료,배달비,광고비,부가세,즉시할인금액,정산_예정_금액,취소금액
0,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,17600.0,4000.0,1373.0,408.0,3400.0,NaN,518.0,0.0,7901.0,NaN
1,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
coupang_df.columns

Index(['collected_at', 'store_id', 'store_name', 'order_date', 'order_id',
       'delivery_type', 'order_status', 'order_summary', 'total_price',
       'is_cancelled', 'menu_name', 'menu_qty', 'menu_price', 'menu_options',
       '매출액', '상점부담_쿠폰', '중개_이용료', '결제대행사_수수료', '배달비', '광고비', '부가세', '즉시할인금액',
       '정산_예정_금액', '취소금액'],
      dtype='object')

In [14]:
# 정산 대상 상태만 남기고 결측치 0으로 채우기
coupang_df = coupang_df[coupang_df["order_status"].isin(["정산예정", "정산완료"])]

# 주요 금액 컬럼 결측치 0 처리 (inplace=False 로 반환값을 할당)
cols_to_zero = [
    "매출액",
    "상점부담_쿠폰",
    "중개_이용료",
    "결제대행사_수수료",
    "배달비",
    "광고비",
    "부가세",
    "즉시할인금액",
]
for col in cols_to_zero:
    if col in coupang_df.columns:
        coupang_df[col] = coupang_df[col].fillna(0)
    else:
        print(f"[WARN] 컬럼 없음: {col}")
        
# 정산 예정 금액 계산: 매출액 - 각종 공제
cond_amt = (
    coupang_df["매출액"]
    - coupang_df["상점부담_쿠폰"]
    - coupang_df["중개_이용료"]
    - coupang_df["결제대행사_수수료"]
    - coupang_df["배달비"]
    - coupang_df["광고비"]
    - coupang_df["부가세"]
    - coupang_df["즉시할인금액"]
)

# 계산 결과도 정수로 변환
cond_amt = cond_amt.fillna(0).astype(int)

# 기존 값이 비어있는 경우에만 계산값으로 채우고 정수로 변환
coupang_df["정산_예정_금액"] = (
    coupang_df["정산_예정_금액"].fillna(cond_amt).astype(int)
)

coupang_df

,collected_at,store_id,store_name,order_date,order_id,delivery_type,order_status,order_summary,total_price,is_cancelled,...,매출액,상점부담_쿠폰,중개_이용료,결제대행사_수수료,배달비,광고비,부가세,즉시할인금액,정산_예정_금액,취소금액
0,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,17600.0,4000.0,1373.0,408.0,3400.0,0.0,518.0,0.0,7901,NaN
1,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN
2,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN
3,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN
4,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN
5,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN
6,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.03 11:19,07CDP6,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17600,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN
7,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.02 21:20,0SEE60,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17000,N,...,17000.0,3000.0,1326.0,420.0,3400.0,0.0,515.0,0.0,8339,NaN
8,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.02 21:20,0SEE60,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17000,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN
9,2026-01-03T03:20:05.294Z,814147,닭도리탕 전문 도리당 가락점,2026.01.02 21:20,0SEE60,배달,정산예정,[한그릇] 누룽지 1인 순살 나만의 백도리당,17000,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN


In [15]:
df = pd.read_csv(r"C:\Users\민준\OneDrive - 주식회사 도리당\Collect_Data\영업팀\baemin_orders_[음식배달]_닭도리탕_전문_도리당_교대점_unknown_20260105.csv")

In [16]:
df.head()

,collected_at,store_name,주문상태,주문번호,주문시각,광고상품,캠페인ID,주문내역,주문수량,결제타입,...,총결제금액,주문옵션상세,주문옵션금액,주문중개,고객할인비용,배달,그외,부가세,만나서결제금액,입금예정금액
0,2026-01-05T00:15:16.714Z,[음식배달] 닭도리탕 전문 도리당 교대점,배달완료,T2990001HMWY,2026. 01. 04. (일) 오후 08:05:01,배민1플러스,12460879,[집밥같은] 도리당 닭도리탕,1,바로결제,...,32000.0,[집밥같은] 도리당 닭도리탕,20800,29504.0,NaN,-4309.0,-960.0,-777.0,NaN,23458.0
1,2026-01-05T00:15:16.714Z,[음식배달] 닭도리탕 전문 도리당 교대점,배달완료,T2990001HMWY,2026. 01. 04. (일) 오후 08:05:01,배민1플러스,12460879,[집밥같은] 도리당 닭도리탕,1,바로결제,...,NaN,(중) 한마리,9000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2026-01-05T00:15:16.714Z,[음식배달] 닭도리탕 전문 도리당 교대점,배달완료,T2990001HMWY,2026. 01. 04. (일) 오후 08:05:01,배민1플러스,12460879,[집밥같은] 도리당 닭도리탕,1,바로결제,...,NaN,순살 닭다리살100% 로 변경,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2026-01-05T00:15:16.714Z,[음식배달] 닭도리탕 전문 도리당 교대점,배달완료,T2990001HMWY,2026. 01. 04. (일) 오후 08:05:01,배민1플러스,12460879,[집밥같은] 도리당 닭도리탕,1,바로결제,...,NaN,[후.참] 계란,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2026-01-05T00:15:16.714Z,[음식배달] 닭도리탕 전문 도리당 교대점,배달완료,T2990001HMWY,2026. 01. 04. (일) 오후 08:05:01,배민1플러스,12460879,[집밥같은] 도리당 닭도리탕,1,바로결제,...,NaN,[후.참] 감자만두,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
import pathlib
baemin_path = pathlib.Path(r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\temp\merged_daily_orders_20260105.parquet")

if baemin_path.exists():
    print(f"✅ 파일 존재: {baemin_path.stat().st_size:,} bytes")
    baemin_df = pd.read_parquet(baemin_path, engine='pyarrow')
    print(f"✅ 로드 성공: {len(baemin_df):,}행")
    baemin_df.head()
else:
    print(f"❌ 파일 없음: {baemin_path}")

❌ 파일 없음: C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\temp\merged_daily_orders_20260105.parquet


In [18]:
df = baemin_df

NameError: name 'baemin_df' is not defined

In [ ]:
df["매장명"] = df["store_name"].str.split(" ").str[-2:].str.join(" ")
df.head()

,platform,order_date,store_id,store_name,ad_id,ad_product,order_id,order_summary,menu_option_name,delivery_type,...,menu_amount,menu_option_price,instant_discount_coupon,instant_discount_coupon_YN,collected_at,option_qty,fee_ad,settlement_amount,instant_discount_amount_YN,매장명
0,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,[집밥같은] 도리당 닭도리탕,알뜰배달,...,32000.0,NaN,1000.0,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점
1,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,(중) 한마리,알뜰배달,...,NaN,NaN,NaN,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점
2,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,순살 닭다리살100% 로 변경,알뜰배달,...,NaN,NaN,NaN,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점
3,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,[후.참] 계란,알뜰배달,...,NaN,NaN,NaN,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점
4,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,[후.참] 감자만두,알뜰배달,...,NaN,NaN,NaN,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점


In [ ]:
employee_df = pd.read_csv(r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\영업팀_DB\sales_employee.csv")
employee_df.head()

,오픈순서,호점,매장명,사업자 번호,점주명,담당자,실오픈일,상세주소,광역,시군구,읍면동,플랫폼,계정ID,계정PW,collected_at,email
0,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,배달의 민족,mjps9142,mjps%0214,2026-01-05 09:10:47,kdk1402@kakao.com
1,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,요기요,mjps9142,mjps$0214,2026-01-05 09:10:47,kdk1402@kakao.com
2,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,쿠팡이츠,mjps9142,mjps$0214,2026-01-05 09:10:47,kdk1402@kakao.com
3,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,땡겨요,mjps9142,mjps%0214,2026-01-05 09:10:47,kdk1402@kakao.com
4,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,토더,doridang100001,abcd9142,2026-01-05 09:10:47,kdk1402@kakao.com


In [ ]:
dfs = pd.merge(df, employee_df, how='left', on='매장명')
dfs.columns

Index(['platform', 'order_date', 'store_id', 'store_name', 'ad_id',
       'ad_product', 'order_id', 'order_summary', 'menu_option_name',
       'delivery_type', 'total_amount', 'menu_amount', 'menu_option_price',
       'instant_discount_coupon', 'instant_discount_coupon_YN',
       'collected_at_x', 'option_qty', 'fee_ad', 'settlement_amount',
       'instant_discount_amount_YN', '매장명', '오픈순서', '호점', '사업자 번호', '점주명',
       '담당자', '실오픈일', '상세주소', '광역', '시군구', '읍면동', '플랫폼', '계정ID', '계정PW',
       'collected_at_y', 'email'],
      dtype='object')

In [ ]:
col = ['platform', 'order_date', 'store_id', 'store_name', 'ad_id',
       'ad_product', 'order_id', 'order_summary', 'menu_option_name',
       'delivery_type', 'total_amount', 'menu_amount', 'menu_option_price',
       'instant_discount_coupon', 'instant_discount_coupon_YN',
       'collected_at_x', 'option_qty', 'fee_ad', 'settlement_amount',
       'instant_discount_amount_YN', '매장명', 
       '담당자', '상세주소', '광역', '시군구', '읍면동','collected_at_y', 'email']

dfs = dfs[col]
dfs.rename(columns={'collected_at_x':'collected_at', 'collected_at_y':'employee_collected_at'}, inplace=True)



NameError: name 'dfs' is not defined

In [12]:
import pandas as pd
df = pd.read_parquet(r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\temp\merged_daily_orders_20260106.parquet")
df.head(2)

,platform,order_date,store_id,store_name,ad_id,ad_product,order_id,order_summary,menu_option_name,delivery_type,total_amount,menu_amount,menu_option_price,instant_discount_coupon,instant_discount_coupon_YN,collected_at,option_qty,fee_ad,settlement_amount,instant_discount_amount_YN
0,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,[집밥같은] 도리당 닭도리탕,알뜰배달,32000.0,32000.0,NaN,1000.0,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None
1,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,(중) 한마리,알뜰배달,NaN,NaN,NaN,NaN,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None


In [21]:
df["stores"] = df["store_name"].str.split(" ").str[-2:].str.join(" ")
df

,platform,order_date,store_id,store_name,ad_id,ad_product,order_id,order_summary,menu_option_name,delivery_type,...,menu_amount,menu_option_price,instant_discount_coupon,instant_discount_coupon_YN,collected_at,option_qty,fee_ad,settlement_amount,instant_discount_amount_YN,stores
0,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,[집밥같은] 도리당 닭도리탕,알뜰배달,...,32000.0,NaN,1000.0,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점
1,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,(중) 한마리,알뜰배달,...,NaN,NaN,NaN,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점
2,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,순살 닭다리살100% 로 변경,알뜰배달,...,NaN,NaN,NaN,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점
3,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,[후.참] 계란,알뜰배달,...,NaN,NaN,NaN,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점
4,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,[후.참] 감자만두,알뜰배달,...,NaN,NaN,NaN,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,쿠팡,2026-01-02 17:39:00,814147.0,닭도리탕 전문 도리당 가락점,NaN,None,191X6V,[한그릇] 누룽지 1인 순살 나만의 백도리당,[한그릇] 누룽지 1인 순살 나만의 백도리당,배달,...,NaN,NaN,0.0,None,2026-01-03 03:20:05.294000+00:00,1.0,0.0,0.0,Y,도리당 가락점
147,쿠팡,2026-01-02 12:52:00,814147.0,닭도리탕 전문 도리당 가락점,NaN,None,00H64F,[한그릇] 누룽지 1인 순살 나만의 백도리당,[한그릇] 누룽지 1인 순살 나만의 백도리당,배달,...,15500.0,NaN,3000.0,None,2026-01-03 03:20:05.294000+00:00,1.0,0.0,7018.0,Y,도리당 가락점
148,쿠팡,2026-01-02 12:52:00,814147.0,닭도리탕 전문 도리당 가락점,NaN,None,00H64F,[한그릇] 누룽지 1인 순살 나만의 백도리당,[한그릇] 누룽지 1인 순살 나만의 백도리당,배달,...,NaN,NaN,0.0,None,2026-01-03 03:20:05.294000+00:00,1.0,0.0,0.0,Y,도리당 가락점
149,쿠팡,2026-01-02 12:52:00,814147.0,닭도리탕 전문 도리당 가락점,NaN,None,00H64F,[한그릇] 누룽지 1인 순살 나만의 백도리당,[한그릇] 누룽지 1인 순살 나만의 백도리당,배달,...,NaN,NaN,0.0,None,2026-01-03 03:20:05.294000+00:00,1.0,0.0,0.0,Y,도리당 가락점


In [41]:
import pandas as pd
path = r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\temp\merged_daily_orders_processed_20260106.parquet"
df = pd.read_parquet(path)
df.head(2)

,platform,order_date,store_id,store_name,ad_id,ad_product,order_id,order_summary,menu_option_name,delivery_type,...,menu_amount,menu_option_price,instant_discount_coupon,instant_discount_coupon_YN,collected_at,option_qty,fee_ad,settlement_amount,instant_discount_amount_YN,stores
0,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,[집밥같은] 도리당 닭도리탕,알뜰배달,...,32000.0,NaN,1000.0,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점
1,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,(중) 한마리,알뜰배달,...,NaN,NaN,NaN,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점


In [48]:
import pandas as pd
path = r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\temp\orders_with_stores_20260106.parquet"
df = pd.read_parquet(path)
df.head(2)

,platform,order_date,store_id,store_name,ad_id,ad_product,order_id,order_summary,menu_option_name,delivery_type,...,플랫폼,계정ID,계정PW,collected_at_y,email,쿠팡_매장명,배민_매장명,composite_key,collection_date,store_names
0,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,[집밥같은] 도리당 닭도리탕,알뜰배달,...,None,None,None,None,None,None,None,None,None,None
1,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,(중) 한마리,알뜰배달,...,None,None,None,None,None,None,None,None,None,None


In [49]:
import pandas as pd
df = pd.read_parquet(r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\temp\merged_daily_orders_processed_20260106.parquet")
df.head(2)

,platform,order_date,store_id,store_name,ad_id,ad_product,order_id,order_summary,menu_option_name,delivery_type,...,menu_amount,menu_option_price,instant_discount_coupon,instant_discount_coupon_YN,collected_at,option_qty,fee_ad,settlement_amount,instant_discount_amount_YN,stores
0,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,[집밥같은] 도리당 닭도리탕,알뜰배달,...,32000.0,NaN,1000.0,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점
1,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,(중) 한마리,알뜰배달,...,NaN,NaN,NaN,Y,2026-01-05 00:15:16.714000+00:00,NaN,NaN,NaN,None,도리당 교대점


In [46]:
import pandas as pd
df = pd.read_parquet(r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\temp\orders_with_stores_processed_20260106.parquet")
df.head(2)

,platform,order_date,store_id,store_name,store_names,ad_id,ad_product,order_id,order_summary,menu_option_name,...,settlement_amount,instant_discount_amount_YN,매장명,담당자,상세주소,광역,시군구,읍면동,employee_collected_at,email
0,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,None,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,[집밥같은] 도리당 닭도리탕,...,NaN,None,None,None,None,None,None,None,None,None
1,배민,2026-01-04 20:05:01,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,None,12460879.0,배민1플러스,T2990001HMWY,[집밥같은] 도리당 닭도리탕,(중) 한마리,...,NaN,None,None,None,None,None,None,None,None,None


In [50]:
df=pd.read_csv(r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\영업팀_DB\sales_employee.csv")

In [ ]:
df.groupby["담당자"]

,오픈순서,호점,매장명,사업자 번호,점주명,담당자,실오픈일,상세주소,광역,시군구,읍면동,플랫폼,계정ID,계정PW,collected_at
0,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,배달의 민족,mjps9142,mjps%0214,2026-01-06 3:29
1,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,요기요,mjps9142,mjps$0214,2026-01-06 3:29
2,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,쿠팡이츠,mjps9142,mjps$0214,2026-01-06 3:29
3,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,땡겨요,mjps9142,mjps%0214,2026-01-06 3:29
4,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,토더,doridang100001,abcd9142,2026-01-06 3:29


In [54]:
df = pd.read_csv(r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\영업팀_DB\sales_employee.csv")

In [59]:
df.head()

,오픈순서,호점,매장명,사업자 번호,점주명,담당자,실오픈일,상세주소,광역,시군구,읍면동,플랫폼,계정ID,계정PW,collected_at,email
0,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,배달의 민족,mjps9142,mjps%0214,2026-01-06 03:33:58,kdk1402@kakao.com
1,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,요기요,mjps9142,mjps$0214,2026-01-06 03:33:58,kdk1402@kakao.com
2,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,쿠팡이츠,mjps9142,mjps$0214,2026-01-06 03:33:58,kdk1402@kakao.com
3,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,땡겨요,mjps9142,mjps%0214,2026-01-06 03:33:58,kdk1402@kakao.com
4,1.0,1,도리당 백석점,203-30-08476,유명진,김덕기 과장,2024-03-18,"경기도 고양시 일산동구 백석동 1418, 지하 1층",경기,고양시,일산동구,토더,doridang100001,abcd9142,2026-01-06 03:33:58,kdk1402@kakao.com


In [69]:
df = df.groupby(['매장명', '담당자', 'email'])["오픈순서"].sum().reset_index()
df[['매장명', '담당자', 'email']]

,매장명,담당자,email
0,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com
1,도리당 강동점,심성준 이사,a17019@kakao.com
2,도리당 경북상주점,이병두 과장,byoungd201@kakao.com
3,도리당 공덕점,김덕기 과장,kdk1402@kakao.com
4,도리당 광주경안점,이병두 과장,byoungd201@kakao.com
...,...,...,...
60,도리당 평택비전점,황대성 대리,gjddkemf@kakao.com
61,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com
62,도리당 하늘도시점,이병두 과장,byoungd201@kakao.com
63,도리당 행신점,김덕기 과장,kdk1402@kakao.com


In [1]:
import pandas as pd

In [15]:
df = pd.read_csv(r"C:\Users\민준\OneDrive - 주식회사 도리당\Doridang_DB\영업관리부_DB\sales_daily_orders.csv")

In [16]:
df

,platform,order_date,store_id,store_name,stores,store_names,ad_id,ad_product,order_id,order_summary,...,settlement_amount,instant_discount_amount_YN,매장명,담당자,상세주소,광역,시군구,읍면동,employee_collected_at,email
0,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,1인 순살 닭도리탕 (밥포함),...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
1,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,1인 순살 닭도리탕 (밥포함),...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
2,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,1인 순살 닭도리탕 (밥포함),...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
3,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,1인 순살 닭도리탕 (밥포함),...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
4,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,1인 순살 닭도리탕 (밥포함),...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23264,쿠팡,2026-01-06 18:03:00,746421.0,닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,NaN,NaN,1661S5,[낙지 곱창 새우] 낙곱새 전골,...,0.0,N,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
23265,쿠팡,2026-01-06 16:10:00,746421.0,닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,NaN,NaN,1FULAF,[1인 메뉴] 순살 닭도리탕,...,16039.0,N,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
23266,쿠팡,2026-01-06 16:10:00,746421.0,닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,NaN,NaN,1FULAF,[1인 메뉴] 순살 닭도리탕,...,0.0,N,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
23267,쿠팡,2026-01-06 16:10:00,746421.0,닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,NaN,NaN,1FULAF,[1인 메뉴] 순살 닭도리탕,...,0.0,N,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com


In [17]:
df.head()

,platform,order_date,store_id,store_name,stores,store_names,ad_id,ad_product,order_id,order_summary,...,settlement_amount,instant_discount_amount_YN,매장명,담당자,상세주소,광역,시군구,읍면동,employee_collected_at,email
0,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,1인 순살 닭도리탕 (밥포함),...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
1,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,1인 순살 닭도리탕 (밥포함),...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
2,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,1인 순살 닭도리탕 (밥포함),...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
3,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,1인 순살 닭도리탕 (밥포함),...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
4,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,1인 순살 닭도리탕 (밥포함),...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com


In [ ]:
import pandas as pd
import hashlib

def add_surrogate_key(df: pd.DataFrame, natural_key_cols: list[str]) -> pd.DataFrame:
    """
    df2 = add_surrogate_key(
        df,
        natural_key_cols=["일자", "주문번호", "시간"]
    )
    df2.head()
    """
    
    # 원본 df 복사 (dtype 절대 변하지 않음)
    out = df.copy()

    # 자연키 컬럼은 key 생성에만 문자열로 변환 (df/out의 dtype은 그대로 유지)
    key_parts = [df[col].astype(str) for col in natural_key_cols]
    uk_series = pd.concat(key_parts, axis=1).agg('|'.join, axis=1)

    # Surrogate key 생성 (해시 16자리)
    out['sub_order_id'] = uk_series.apply(
        lambda s: hashlib.sha1(s.encode('utf-8')).hexdigest()[:16]
    )

    # key 컬럼을 맨 앞으로 이동
    cols = ['sub_order_id'] + [c for c in out.columns if c != 'sub_order_id']
    out = out[cols]

    return out

df = add_surrogate_key(df, natural_key_cols=["order_id", "collected_at", "order_summary"])
df_col = ['platform', 'order_date', 'store_id', 'store_name', 'stores',
       'store_names', 'ad_id', 'ad_product', 'order_id','sub_order_id', 'order_summary',
       'menu_option_name', 'delivery_type', 'total_amount', 'menu_amount',
       'menu_option_price', 'instant_discount_coupon',
       'instant_discount_coupon_YN', 'collected_at', 'option_qty', 'fee_ad',
       'settlement_amount', 'instant_discount_amount_YN', '매장명', '담당자', '상세주소',
       '광역', '시군구', '읍면동', 'employee_collected_at', 'email']

df = df[df_col]

In [27]:
df

,platform,order_date,store_id,store_name,stores,store_names,ad_id,ad_product,order_id,sub_order_id,...,settlement_amount,instant_discount_amount_YN,매장명,담당자,상세주소,광역,시군구,읍면동,employee_collected_at,email
0,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,80ae04751cee58e8,...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
1,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,80ae04751cee58e8,...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
2,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,80ae04751cee58e8,...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
3,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,80ae04751cee58e8,...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
4,배민,2025-08-31 20:14:52,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T25R0001LJDO,80ae04751cee58e8,...,NaN,NaN,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23264,쿠팡,2026-01-06 18:03:00,746421.0,닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,NaN,NaN,1661S5,522c73ba4686f2c1,...,0.0,N,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
23265,쿠팡,2026-01-06 16:10:00,746421.0,닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,NaN,NaN,1FULAF,402fbca216202a42,...,16039.0,N,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
23266,쿠팡,2026-01-06 16:10:00,746421.0,닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,NaN,NaN,1FULAF,402fbca216202a42,...,0.0,N,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
23267,쿠팡,2026-01-06 16:10:00,746421.0,닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,NaN,NaN,1FULAF,402fbca216202a42,...,0.0,N,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-07 02:07:07,a17019@kakao.com
